In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed) 
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "BiasedMNIST",
            'fairness_agg': 'mean',
            'model': 'resnet18',

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 128,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 5,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:6' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha': 0.00,
            'lambda': 1.0,
            'lambda_old': 0.0,
              }
    


#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_tau={params['tau']}_alpha={params['alpha']}"
    if params['lambda'] != 0:
        trial_id+=f"_lmbd_{params['lambda']}_lmbdold_{params['lambda_old']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=BiasedMNIST/seed=10_epoch=1_lr=0.001_tau=5_alpha=0.0_lmbd_1.0_lmbdold_0.0


In [2]:
"MNIST" in params['dataset']

True

In [3]:
from datasets import BiasedMNIST

if  params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                                per_task_memory_examples=params['per_task_memory_examples'],
                                per_task_examples = params['per_task_examples'],
                                random_class_idx=True)
    input_dim = (3, 28, 28)
    class_idx = benchmark.class_idx
    num_classes = len(class_idx)

[8 2 5 6 3 1 0 7 4 9]


In [4]:
from torchvision.transforms.functional import to_pil_image
from cl_gym.benchmarks.transforms import MNIST_MEAN, MNIST_STD
COLOR_MAP = {
    0: (1, 0, 0),
    1: (0, 1, 0),
    2: (1, 1, 0),
    3: (0, 0, 1),
    4: (1, 0.65, 0),
    5: (0.5, 0, 0.5),
    6: (0, 1, 1),
    7: (1, 0.75, 0.8),
    8: (0.8, 1, 0),
    9: (.588, .294, 0.)
}

r = (1 - 0.1913)
color_values = np.array(list(COLOR_MAP.values()))
m_rgb = color_values.mean(axis=0)
std_rgb = color_values.std(axis=0)

bmnist_mean = [r*m + MNIST_MEAN[0] for m in m_rgb]
bmnist_std = [(r*s**2+(1-r)*MNIST_STD[0]**2+r*(1-r)*(bmnist_mean[i] - m_rgb[i])**2)**0.5 for i, s in enumerate(std_rgb)]

unnormalize = torchvision.transforms.Normalize([-m/s for m, s in zip(bmnist_mean, bmnist_std)], [1/s for s in bmnist_std])
sample_idx = 95
to_pil_image(unnormalize(benchmark.trains[2][sample_idx][0]), mode="RGB")

In [5]:
target_label = 0
incremental_step = 1
# cat_img = torch.cat([img for img in benchmark.trains[incremental_step].inputs[benchmark.trains[incremental_step].targets == target_label][20:30]], dim=2)
cat_img = torch.cat([img for img, target, *_ in benchmark.tests[incremental_step]][20:30], dim=2)
to_pil_image(unnormalize(cat_img), mode="RGB")

In [6]:
from trainers import FairContinualTrainer
from trainers.fair_trainer import FairContinualTrainer2
from metrics import FairMetricCollector
from algorithms import Heuristic3
from algorithms.fairl import FaIRL
from backbones import MLP2Layers2, ResNet18Small2

# backbone = MLP2Layers2(input_dim=n_feature, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
backbone = ResNet18Small2(
        input_dim=input_dim, 
        output_dim=num_classes,
        class_idx=class_idx,
        config=params
    ).to(params['device'])
algorithm = Heuristic3(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = FairMetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

trainer = FairContinualTrainer2(algorithm, params, callbacks=[metric_manager_callback])


In [7]:
if params['fairness_agg'] == "mean":
    agg = np.mean
elif params['fairness_agg'] == "max":
    agg = np.max
else:
    raise NotImplementedError

metric_manager_callback.meters['fairness'].agg = agg
metric_manager_callback.meters['fairness'].agg


<function mean at 0x7f0adb427c70>

In [8]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 77.34269296276841, 'loss': 0.0023766214386416098, 'fairness': [0.3360995850622407, 0.5595463137996219], 'accuracy_s0': 100.0, 'accuracy_s1': 55.217705056906865, 'classwise_accuracy': {8: array([812, 974]), 2: array([ 736, 1032])}}
training_task_end
---------------------------- Task 2 -----------------------
[2] Eval metrics for task 1 >> {'accuracy': 92.77862009136781, 'loss': 0.0014892295820049845, 'fairness': [0.13697162905239002, 0.13780079447699434], 'accuracy_s0': 99.69956682668217, 'accuracy_s1': 85.96094565021296, 'classwise_accuracy': {8: array([906, 974]), 2: array([ 955, 1032])}}
[2] Eval metrics for task 2 >> {'accuracy': 66.88226125055, 'loss': 0.005384430240940403, 'fairness': [0.8009084885261909, 0.5261871772263509], 'accuracy_s0': 99.14765467346956, 'accuracy_s1': 32.79287138584247, 'classwise_accuracy': {6: array([578, 958]), 5: array([655, 892])}}
training_task_end
-

In [9]:
metric_manager_callback.meters['accuracy'].get_data()

array([[77.343,  0.   ,  0.   ,  0.   ,  0.   ],
       [92.779, 66.882,  0.   ,  0.   ,  0.   ],
       [74.584, 83.172, 92.955,  0.   ,  0.   ],
       [70.219, 69.842, 91.3  , 93.171,  0.   ],
       [65.846, 67.909, 83.656, 92.348, 90.244]])

In [10]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

80.3753615176966

In [12]:
[np.round(x, 3) for x in metric_manager_callback.meters['fairness'].compute_overall()]

[0.448, 0.4, 0.174, 0.201, 0.205]

In [13]:
np.mean(metric_manager_callback.meters['fairness'].compute_overall())

0.28562128509612755

In [14]:
task_id = 2

print(f"{task_id=}")
print(f"sensitive samples / all samples = {(benchmark.trains[task_id].sensitives != benchmark.trains[task_id].targets).sum().item()} / {benchmark.trains[task_id].sensitives.shape[0]}")

updated_seq_indices = benchmark.seq_indices_train[task_id]
print(f"sensitive samples / selected samples = {(benchmark.trains[task_id].sensitives[updated_seq_indices] != benchmark.trains[task_id].targets[updated_seq_indices]).sum().item()} / {len(updated_seq_indices)}")


task_id=2
sensitive samples / all samples = 595 / 11339
sensitive samples / selected samples = 530 / 10000


In [15]:
step_class = 2
for i in range(2, 6):
    incremental_step = i
    print(f"{incremental_step=}")
    one_idx = benchmark.trains[incremental_step].sample_weight > 0.9

    print(f"{2*i-2} : {(benchmark.trains[incremental_step].targets == (2*i-2)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-2)).sum().item()}")
    print(f"{2*i-1} : {(benchmark.trains[incremental_step].targets == (2*i-1)).sum().item()} --> {(benchmark.trains[incremental_step].targets[one_idx] == (2*i-1)).sum().item()}")


incremental_step=2
2 : 0 --> 0
3 : 0 --> 0
incremental_step=3
4 : 0 --> 0
5 : 0 --> 0
incremental_step=4
6 : 0 --> 0
7 : 6265 --> 6265
incremental_step=5
8 : 0 --> 0
9 : 5949 --> 5949


In [16]:
for eos in metric_manager_callback.meters['fairness'].get_data():
    for eo in eos:
        print(round(eo, 3),end=" ")
    print()

0.336 0.56 
0.137 0.138 0.801 0.526 
0.13 0.16 0.196 0.31 0.126 0.12 
0.125 0.168 0.496 0.327 0.135 0.099 0.164 0.092 
0.173 0.169 0.229 0.463 0.175 0.192 0.103 0.158 0.137 0.252 


In [17]:
metric_manager_callback.meters['classwise_accuracy'].get_data()

[{8: array([812, 974]), 2: array([ 736, 1032])},
 {8: array([906, 974]),
  2: array([ 955, 1032]),
  6: array([578, 958]),
  5: array([655, 892])},
 {8: array([729, 974]),
  2: array([ 767, 1032]),
  6: array([816, 958]),
  5: array([724, 892]),
  3: array([ 946, 1010]),
  1: array([1047, 1135])},
 {2: array([ 732, 1032]),
  8: array([677, 974]),
  5: array([597, 892]),
  6: array([697, 958]),
  3: array([ 909, 1010]),
  1: array([1051, 1135]),
  7: array([ 939, 1028]),
  0: array([931, 980])},
 {8: array([639, 974]),
  2: array([ 682, 1032]),
  6: array([704, 958]),
  5: array([556, 892]),
  1: array([ 991, 1135]),
  3: array([ 808, 1010]),
  0: array([912, 980]),
  7: array([ 942, 1028]),
  4: array([914, 982]),
  9: array([ 882, 1009])}]